In [1]:
import matplotlib.pyplot as plt
from core import MontecarloBlackjack, DummyAgent, NeuralNetworkAgent, QLearningAgent
from juego import Juego
from tqdm import tqdm

2024-01-05 01:38:15.117509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 01:38:15.170653: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 01:38:15.171749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 01:38:16.260118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Q-Learning

In [2]:
def obtener_nuevo_estado(juego, accion):
    if accion == [1,0]:
        juego.jugador.actualizar_mano(juego.baraja.pop(0))
        return juego.estado.get_estado_codificado()
        
    elif accion == [0,1]:
        return juego.estado.get_estado_codificado()

In [3]:
def train(episodios):
    agente = QLearningAgent()
    resultados = []

    for _ in tqdm(range(episodios), desc="Entrenando Agente"):
        juego = Juego(agente)
        juego.setup()
        while not juego.is_terminal():
            if len(juego.baraja) < 1:
                juego.baraja = juego.new_deck()
            jugador_activo = juego.jugador_activo()
            estado_codificado = tuple(juego.estado.get_estado_codificado())
            accion = agente.get_action(estado_codificado)
            juego.realizar_accion(jugador_activo, accion)
        resultado_episodio = juego.get_resultados()
        if resultado_episodio:
            resultados.extend(resultado_episodio)
            for estado_codificado, accion, res in resultado_episodio:
                accion_str = 'HIT' if accion == [1, 0] else 'STAND'
                nuevo_estado = obtener_nuevo_estado(juego, accion)
                agente.update_q_table(estado_codificado, accion_str, res, nuevo_estado)
    return agente, resultados

In [4]:
agente, resultados = train(100)

Entrenando Agente:  25%|██▌       | 25/100 [00:00<00:00, 9567.30it/s]


Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Acciones posibles: ['HIT', 'STAND'], Acción intentada: STAND
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Acciones posibles: ['HIT', 'STAND'], Acción intentada: HIT
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Valor jugador: 15, Valor crupier: 17, Resultado: -1
Valor jugador: 12, Valor crupier: 17, Resultado: -1
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Acciones posibles: ['HIT', 'STAND'], Acción intentada: HIT
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Acciones posibles: ['HIT', 'STAND'], Acción intentada: HIT
Estado terminal: False, Acciones disponibles: ['HIT', 'STAND']
Valor jugador: 23, Valor crupier: 14, Resultado: -1
Valor jugador: 13, Valor crupier: 19, Resultado: -1
Valor jugador: 19, Valor crupier: 18, Resultado: 1

ValueError: Acción inválida

In [ ]:
def plot_ganancias(resultados):
    ganancias_acumuladas = [sum(r[-1] for r in resultados[:i+1]) for i in range(len(resultados))]
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(ganancias_acumuladas) + 1), ganancias_acumuladas)
    plt.title("Ganancias Acumuladas Durante el Entrenamiento")
    plt.xlabel("Episodio")
    plt.ylabel("Ganancias Acumuladas")
    plt.show()

plot_ganancias(resultados) 

In [ ]:
ganancias = sum([r[-1] for r in resultados])
ganancias

In [ ]:
# Suponiendo que tienes la clase Juego y las clases de agentes definidas
dummy_agent = DummyAgent()
neural_network_agent = NeuralNetworkAgent(28)

simulador = MontecarloBlackjack({'DummyAgent': dummy_agent, 'NeuralNetworkAgent': neural_network_agent}, 6, False)
simulador.simular()
simulador.plot_resultados()